In [ ]:
"""
This script contains usefull functions used in the PLAID_to_IDOT tools

@author: Jonne Rietdijk
"""

In [81]:
def log_10_dilutions(max_mM):
    """ 
    This function creates 10 log doses starting from the highest given dose
    e.g. give 100 returns 100,10,1,0.1,0.01...0.00000000001
  
    """    
    import math 
    availstocks_mM = []  
    
    maxlog10 = int(math.log(max_mM,10))
    
    for i in range(-12, maxlog10 + 1):
        availstocks_mM.append(pow(10, i))
    availstocks_mM.append(max_mM)
        
    return availstocks_mM

def stockfinder(concUM, highest_stock_mM, V2_ul, dmso_percmax, sourceplate_type):  
    
    if highest_stock_mM != 0:
        availstocks_mM = log_10_dilutions(highest_stock_mM)

        if sourceplate_type == "S.200":
          MinV1_nl = 30 
        else:
          MinV1_nl = 8

        MaxV1_nl = (dmso_percmax / 100) * (V2_ul*1000)                                                                  

        C1_low  = (V2_ul * concUM) / MaxV1_nl                                                  
        C1_high = (V2_ul * concUM) / MinV1_nl                                                 

        psblstocks = [x for x in availstocks_mM if x >= C1_low and x <= C1_high]    
        
        if psblstocks:
            highestStock = max(psblstocks)   # select highest stock for your condition                                            
            return highestStock    
        else:
            raise Exception("not possible to find a suitable stock for requested settings")
    else:
        return 0 

In [72]:
def treatmentdict(cmpdname):
    import re
    
    DMSO, ctrls, blank, trt = ([] for i in range(4))
    catdict = {}
    
    for i in cmpdname:
        if bool(re.search('.*dmso.*', i,re.IGNORECASE)):
            findDMSO = re.findall('.*dmso.*', i, re.IGNORECASE) 
            DMSO.append(i)
            if findDMSO[0] not in catdict:
                catdict[i] = "DMSO" 
            

        elif bool(re.search('.*blank.*', i,re.IGNORECASE)):
            findblank = re.findall('.*blank.*', i, re.IGNORECASE) 
            blank.append(i)
            if findblank[0] not in catdict:
                catdict[i] = "blank" 
            

        elif bool(re.search(r'\[.*?\]', i)):
            findctrl = re.findall(r'\[[a-zA-Z0-9_]{4}?\]', i) 
            ctrls.append(i)
            if findctrl[0] not in catdict:
                catdict[i] = "ctrl" 
            
        else:
            if i not in catdict:
                trt.append(i)
                catdict[i] = "trt" 
    return catdict

In [15]:
def createplate(size, direction):

    import string

    if size == 96:
        colr = 13
        rowr = 8
    
    if size == 384:
        colr = 25
        rowr = 16
    

    row = list(string.ascii_uppercase[:rowr])
    col = [(f'{i:02d}') for i in range(1, colr, 1)]
    wells = []
      
    if direction == "vert":
        for c in col:
            for r in row:
                wells.append(str(r+c))
        return(wells)
    
    else:
        for r in row:
            for c in col:
                wells.append(str(r+c))
        return(wells)

In [1]:
def normalizeDMSO(df,maxDMSO):
    dfDMSO = df.copy()
    dfDMSO["DMSO_backfill_uL"] = maxDMSO - dfDMSO["Volume [uL]"]
    dfDMSO["DMSO_backfill_uL"][dfDMSO["DMSO_backfill_uL"] < 0] = 0
    dfDMSO                     = dfDMSO[dfDMSO.DMSO_backfill_uL != 0] 
    dfDMSO.drop(["Volume [uL]"], axis=1)
    dfDMSO["Volume [uL]"]      = dfDMSO["DMSO_backfill_uL"] 
    
    dfDMSO[["Liquid Name","cmpdname","treatment_type"]]   = "DMSO"
    dfDMSO[["highest_stock_mM","stock_conc_mM"]]          = 0
    dfDMSO = dfDMSO[["Target Plate","cmpdname","highest_stock_mM","stock_conc_mM","treatment_type","Target Well","Liquid Name","Volume [uL]"]]
    return dfDMSO
    

In [23]:
def uLfromstock(concUM,stock_conc_mM,V2_ul):
    concUM =  (concUM * V2_ul) / stock_conc_mM if stock_conc_mM != 0 else 0
    return concUM / 1000

In [18]:
def stockhighestmM(treatment_type,maxmM_treat,maxmM_ctrl):
    if treatment_type == "trt":
        return maxmM_treat
    if treatment_type == "ctrl":
        return maxmM_ctrl
    if treatment_type == "DMSO":
        return 0
    if treatment_type == "blank":
        return 0

In [19]:
def removeleadingzero(x):
    x = x[0] + x[1:3].lstrip("0")
    return x 

In [12]:
def assignsource_automated(df2,treatment_type):
  
    if len(df2[df2["treatment_type"]==str(treatment_type)]) == 0:
        print("OBS: no", treatment_type, "found")

    else:
        print(treatment_type)
        seldf                     = df2[df2["treatment_type"]==str(treatment_type)] # make subset of treatment type
        seldf["stock_conc_mM"]    = seldf.stock_conc_mM.astype(float)
        seldf["dilutions1in10"]   = np.log10(seldf["highest_stock_mM"].astype(int) / seldf["stock_conc_mM"]).astype(int) + 1 # how many dilution steps are needed
        seldf                     = seldf.groupby(["Liquid Name"])[["cmpdname","stock_conc_mM","dilutions1in10","treatment_type"]].max().reset_index()
        maxdils     = seldf["dilutions1in10"].max() # max number of dilutions in this treatment type
        print("max number of dilutions in treatment group:",maxdils)
        compounds   = np.unique(seldf[["cmpdname"]].values).tolist()
        compounds   = sorted(compounds)
        nrcompounds = len(compounds)
        print("total number of compounds:", nrcompounds)


        # ----------------- source plates ------------------#
        nrsubplates        = math.floor(12 / maxdils)    # calculate how many subplates per 96 well source plate
        maxcmpdperplate    = nrsubplates * 8             # how many compounds do fit on one 96 well plate?
        totalsubplates     = math.ceil(nrcompounds / 8)  # total number of subplates
        print("total number of subplates:", totalsubplates)
        print("number of subplates per 96 well plate:", nrsubplates)
        plates             = math.ceil(nrcompounds / maxcmpdperplate)  # total number of source plates
        plates             = int(plates)
        print("total number of source plates:", plates)

        # --------------- create plates and subplates------#
        x         = int(math.floor(12/ nrsubplates)) # start position each subplate
        startcol  = [int(f"{i:01d}") for i in range(1, 13, x)] * plates
        startcols = startcol[0:totalsubplates]
        row96     = list(string.ascii_uppercase[:8])
        welldict = {}
        #print(startcols)

        for i, cmp in enumerate(compounds):
            for j, dilution in enumerate(range(0,maxdils)):
                comp = i+1                                                  # counts chemicals starting at 1
                subplate = math.floor(comp/8)                               # decides on subplate (e.g. compound 20 will be on 3th subplate)
                subplateindex = subplate - 1                                # starts counting subplates at zero
                plate = math.floor(subplate / nrsubplates)                  # source plate number
                plate  = treatment_type + "_" + str(plate + 1)              # name of plate
                sub  = treatment_type + "_" + str(subplate)                 # name of subplate

                # assigns row letter for compound
                #cmprow = str(cmprow[i])   
                cmprow = (list(string.ascii_uppercase[:8]) * totalsubplates)[0:len(compounds)] # generated list of letters (A-H) for the number of subplates 
                cmprow = str(cmprow[i]) 
                #print(cmprow) 
                                                                
                # assign column number for concentration
                cmpcol = startcols[subplate]
                cmpcol = cmpcol + j
                cmpcol = str(cmpcol).zfill(2)
                #print(cmpcol)

                # configures well location
                well = cmprow+cmpcol
                #print(well)

                # dictionary of compound, well, dilution and platename
                if seperate_batches_8 == True:
                  welldict[i,j] = [well,cmp,j+1,sub]
                else:
                  welldict[i,j] = [well,cmp,j+1,plate]

        source      = pd.DataFrame.from_dict(welldict,orient="index", columns=["Source Well","cmpdname","dilutions1in10", "Source Plate"])
        sourceplate = pd.merge(seldf, source,  how="left", left_on=["cmpdname","dilutions1in10"], right_on = ["cmpdname","dilutions1in10"])
        return(sourceplate)

In [15]:
def assign_DMSOsource(max_volume, dfvolumes):
    
    
    wellcapacity = int(max_volume *1e06) * 0.8 # wellcapacity based on idot plate (but with 20% safety margin)
    well_state = {"well_number": 0, "current_amount": wellcapacity}
    DMSOwells = createplate(size=96,direction="vert")
    
    sourcewelllist = []

    for volume in dfvolumes:
        remaining = well_state["current_amount"] - volume
        if remaining < 0:
            well_state["well_number"] += 1
            well_state["current_amount"] = wellcapacity
            wellindex = well_state["well_number"]
            sourcewelllist.append(DMSOwells[wellindex])
        else:
            well_state["current_amount"] -= volume
            wellindex = well_state["well_number"]
            sourcewelllist.append(DMSOwells[wellindex])
            
    sourcewells = [*set(sourcewelllist)]      
    return sourcewelllist

In [14]:
def treatmentstodict(conditions):
    import re
    
    DMSO, ctrls, blank, trt = ([] for i in range(4))
    catdict = {}
    
    for i in conditions:
        if bool(re.search('.*dmso.*', i,re.IGNORECASE)):
            findDMSO = re.findall('.*dmso.*', i, re.IGNORECASE) 
            DMSO.append(i)
            if findDMSO[0] not in catdict:
                catdict[i] = "DMSO" 
            

        elif bool(re.search('.*blank.*', i,re.IGNORECASE)):
            findblank = re.findall('.*blank.*', i, re.IGNORECASE) 
            blank.append(i)
            if findblank[0] not in catdict:
                catdict[i] = "blank" 
            

        elif bool(re.search(r'\[.*?\]', i)):
            findctrl = re.findall(r'\[[a-zA-Z0-9_]{4}?\]', i) 
            ctrls.append(i)
            if findctrl[0] not in catdict:
                catdict[i] = "ctrl" 
            
        else:
            if i not in catdict:
                trt.append(i)
                catdict[i] = "trt" 
            
    print("A total of       ",len(conditions),"conditions were found","\n")
    print("blanks           ",len(blank),"blank control(s) are identified in your assay:\n                 ", blank,"\n")
    print("DMSO:            ",len(DMSO), "DMSO control(s) are identified in your assay:\n                 ",DMSO,"\n")
    print("Controls:        ",len(ctrls), "control compounds are identified in your assay:\n                 ", ctrls,"\n")
    print("Treatments:      ",len(trt), "treatments are identified in your assay:\n                 ", trt,"\n")
    
    return catdict


In [ ]:
def assignsourcesimple(dfs): # input list of dataframes to be included

    alldfs             = pd.concat(dfs)
    compounds          = alldfs["Liquid Name"].unique().tolist()
    nrcompounds        = len(compounds)
      
    nrplates           = int(math.ceil(nrcompounds / 96))                                             
    wells              = createplate(size = 96, direction= "vert") * nrplates # generate all wells of the number of plates needed
    wells              = wells[0:nrcompounds]     

    
    welldict = {}
    counter = 96 

    for i, cmpd in enumerate(compounds):
      platenumber = 1
      remaining = counter - i

      if remaining < 0:
          platenumber += 1
      else:
          sourceplate = "sourceplate"+ str(platenumber)
          welldict[i] = [wells[i], cmpd, sourceplate]  
  
    allsourceplates      = pd.DataFrame.from_dict(welldict,orient='index', columns=['Source Well','cmpdname','Source Plate'])

    return allsourceplates